# Sorting puzzle game






### Idea

**Goal :** 
* To sort the balls by colour in each of the n stacks.

**Rules:**
* Only the balls present in the top of the stack can be moved.
* A ball can be moved on top of the another ball of same color.
* A ball can be moved in an empty stack.

**Implementation idea:**
* Each column represents a stack
* Numbers represent the color of the balls.


### A* algorithm

In [4]:
import numpy as np
from itertools import groupby
from itertools import permutations

In [2]:
class Node:

  def __init__(self,data,level,fval):
    self.data=data
    self.level=level
    self.fval=fval
    self.rows=len(data)
    self.cols=len(data[0])
  
  def getTopEle(self,column):
    i=0
    while i<self.rows and column[i]=='-':
      i+=1
    if i==self.rows:
      return -1
    else:
      return i
  
  def copy(self,root):
    temp=[]
    for i in root:
      t=[]
      for j in i:
        t.append(j)
      temp.append(t)
    return temp

  def shuffle(self,root,noOfEle,srcIndex1,srcIndex2,destIndex1,destIndex2):
    temp_puz=[]
    temp_puz=self.copy(root)
    # print(temp_puz)
    # print(srcIndex1,srcIndex2)
    for i in range(0,noOfEle):
      temp=temp_puz[srcIndex1][srcIndex2]
      temp_puz[srcIndex1][srcIndex2]=temp_puz[destIndex1][destIndex2]
      temp_puz[destIndex1][destIndex2]=temp
      srcIndex1-=1
      destIndex1+=1

    return temp_puz

  def printMatrix(self,data):
    for i in data:
      for j in i:
        print(j,end=" ")
      print("\n")

  def gettingCountOfSimiliarEle(self,column,topEle,pos):
    count=0
    for i in range(pos,self.rows):
      if(column[i]==topEle):
        count+=1
      else:
        return count
    return count

  def generate_child(self):
    children=[]
    col=list(zip(*self.data))
    for i in range(0,self.cols):
      topElePos=self.getTopEle(col[i])
      # print("For {}".format(i))
      if topElePos!=-1:
        numberOfEle=self.gettingCountOfSimiliarEle(col[i],col[i][topElePos],topElePos+1)+1
        # print(numberOfEle)
        for j in range(0,self.cols):
          if i!=j:
            k=0
            while k<self.rows and col[j][k]=='-':
              k=k+1
            if k!=0 and (k==self.rows or col[j][k]==col[i][topElePos]):
              if k>=numberOfEle:
                child=self.shuffle(self.data,numberOfEle,k-1,j,topElePos,i)
              else:
                child=self.shuffle(self.data,k,k-1,j,topElePos,i)
              child_node=Node(child,self.level+1,0)
              children.append(child_node)
    return children
              
        

In [12]:
class Puzzle:
  def __init__(self):
    self.n=0
    self.open=[]
    self.closed=[]
    self.optimalMoves=[]

  def h(self,start):
    col=list(zip(*start.data))
    count_similiar=0
    for i in range(0,self.n):
      count_similiar += len([sum(1 for _ in group) for _, group in groupby(col[i]) if _!='-'])
    return count_similiar
  
  def f(self,start):
    return start.level+self.h(start)
  
  def returnBest(self,lists):
    countList=[]
    for i in lists:
      count=0
      cols=list(zip(*i.data))
      for j in range(0,self.n):
        temp_list = [x for x in cols[j] if x != '-']
       
        temp=set(temp_list)
        temp=list(temp)
        count+=(len(temp_list)+1-len(temp))
      countList.append(count)
    max_index=countList.index(max(countList))
    return max_index
    # print(max_index)
    # lists[max_index].printMatrix(lists[max_index].data)
    # print(countList)

  def accept(self):
    puz=[['-','1','-','-','-','-'],
         ['2','3','-','4','-','4'],
         ['2','3','-','2','3','4'],
         ['1','2','1','1','3','4']
         ]
    self.n=len(puz[0])
    # for i in range(0,self.n):
    #   temp = input().split(" ")
    #   puz.append(temp)
    temp=[]
    count=0
    for d in puz:
      for ele in d:
        if ele!='-' and ele not in temp:
          count+=1
          temp.append(ele)
    return puz,count

  def checkingMatrix(self,cur,data):
    cols=list(zip(*cur))
    permu=list(permutations(cols))
    for i in permu:
      if any(np.array_equal(i,list(zip(*x.data))) for x in data) == True:
        return True
    return False
  
  def printOptimalWays(self):
    for x in self.optimalMoves:
      x.printMatrix(x.data)
      print("-------------")
      
  def getOptimalNoOfMoves(self):
    start,noOfColors=self.accept()
    start=Node(start,0,0)
    start.fval=self.f(start)
    # print(start.fval)
    self.open.append(start)
    moves=-1
    # print("Number of colors :" ,noOfColors)
    while True:
        moves+=1
        cur = self.open[0]
        self.open = []
        self.optimalMoves.append(cur)
        # cur.printMatrix(cur.data)
        if(self.h(cur) == noOfColors):
          # print("No of moves:",moves)
          return moves
        d=cur.generate_child()
        for i in d:
          i.fval = self.f(i)
          # Checking in closed list
          if self.checkingMatrix(i.data,self.closed)==False and any(np.array_equal(i.data,x.data) for x in self.closed) == False:
            self.open.append(i)
        self.closed.append(cur)
        del self.open[0]
        self.open.sort(key = lambda x:x.fval,reverse=False)
        first_fval=self.open[0].fval
        lists=[i for i in self.open if i.fval==first_fval]
        max_index=self.returnBest(lists)
        temp=self.open[0]
        self.open[0]=self.open[max_index]
        self.open[max_index]=temp
        # print("Next child")
        # self.open[0].printMatrix(self.open[0].data)
        # print(self.open[0].fval)
        # print("---------------------------------------------------------------")


In [13]:
puz=Puzzle()
print("Number of moves:",puz.getOptimalNoOfMoves())
puz.printOptimalWays()

Number of moves: 7
- 1 - - - - 

2 3 - 4 - 4 

2 3 - 2 3 4 

1 2 1 1 3 4 

-------------
- 1 - - - 4 

2 3 - - - 4 

2 3 - 2 3 4 

1 2 1 1 3 4 

-------------
- - - - - 4 

2 3 - - - 4 

2 3 1 2 3 4 

1 2 1 1 3 4 

-------------
- - - - 3 4 

2 - - - 3 4 

2 - 1 2 3 4 

1 2 1 1 3 4 

-------------
- - - 2 3 4 

- - - 2 3 4 

- - 1 2 3 4 

1 2 1 1 3 4 

-------------
- - - 2 3 4 

1 - - 2 3 4 

1 - - 2 3 4 

1 2 - 1 3 4 

-------------
- 2 - - 3 4 

1 2 - - 3 4 

1 2 - - 3 4 

1 2 - 1 3 4 

-------------
- 2 - 1 3 4 

- 2 - 1 3 4 

- 2 - 1 3 4 

- 2 - 1 3 4 

-------------


### Puzzle Game

***Steps:***
1. Initialize the start state (The start state can be generated randomly or we can have collection of start states and we can choose one among them as start state)
2. Get the  inputs from user ( from where to where)
3. Validate the inputs (is it possible to move or not?)
4. Once the move has done, Check if the game is over. If so, display the number of moves
5. Check for other cases also (out of moves)
6. If the player wins but the player can't solve the game in the best steps, print the optimal steps.

In [17]:

numberOfColors=4
numberOfRows=4
numberOfCols=6

def convertColumnWise(state):
  col=list(zip(*state))
  return col

def getTopElement(column):
    i=0
    while i<numberOfRows and column[i]=='-':
      i+=1
    if i==4:
      return -1,-1
    else:
      return [i,column[i]]

def findNumberOfUniqueValues(column):
  temp=set(column)
  temp=list(temp)
  return len(temp)

def display(state):
  for i in state:
    for j in i:
      print(j,end=" ")
    print()

def doesPlayerwin(state):
  col=convertColumnWise(state)
  count=0
  for i in range(0,numberOfCols):
    temp_list = [x for x in col[i] if x != '-']
    count+=findNumberOfUniqueValues(temp_list)
  if count==numberOfColors:
    return True
  else:
    return False

def checkIfMoveIsPossible(stateColumnWise,fromCol,toCol):
  '''
  FromCol:
    May be empty
  ToCol:
    May be full with numbers
  FromCol and ToCol don't have same colors on top
  and ToCol doesn't have empty spaces.

  '''
  srcCol=stateColumnWise[fromCol]
  destCol=stateColumnWise[toCol]
  # print(srcCol)
  # print(destCol)
  if findNumberOfUniqueValues(destCol)==1 and destCol[0]=='-':
    return True
  elif (findNumberOfUniqueValues(srcCol)==1 and srcCol[0]=='-') or (findNumberOfUniqueValues(destCol)==1 and destCol[0]!='-'):
    return False
  elif stateColumnWise[toCol][0]!='-':
    return False
  elif (getTopElement(srcCol))[1]!=(getTopElement(destCol))[1]:
    return False
  else:
    return True 

def getNumberOfBlankSpacesInDestCol(destCol):
  i=0
  while i<numberOfRows and destCol[i]=='-':
    i+=1
  return i

def getNumberOfBallsToMove(srcColumn,topEle,pos):
  count=0
  for i in range(pos,4):
    if(srcColumn[i]==topEle):
      count+=1
    else:
      return count
  return count

  
def makeDuplicate(root):
  temp=[]
  for i in root:
    t=[]
    for j in i:
      t.append(j)
    temp.append(t)
  return temp

def makeMove(currentState,noOfEle,srcIndex,fromColumn,destIndex,toColumn):
  next_state=[]
  next_state=makeDuplicate(currentState)
  for i in range(0,noOfEle):
    temp=next_state[srcIndex][fromColumn]
    next_state[srcIndex][fromColumn]=next_state[destIndex][toColumn]
    next_state[destIndex][toColumn]=temp
    srcIndex+=1
    destIndex-=1
  return next_state

def outOfMoves(currentState):
  columns=convertColumnWise(currentState)
  checkCols=[]
  for i in columns:
    if findNumberOfUniqueValues(i)==1 and i[0]=='-':
      return False
    else:
      checkCols.append(getTopElement(i))
  for i in checkCols:
    for j in checkCols:
      if i!=j:
        if i[1]==j[1] and (i[0]>0 or j[0]>0):
          return False
  return True


def sortPuzzle():
  moveCount=0
  state=[['-','1','-','-','-','-'],
         ['2','3','-','4','-','4'],
         ['2','3','-','2','3','4'],
         ['1','2','1','1','3','4']
         ]
  # numberOfColors=4
  # numberOfRows=len(state)
  # numberOfCols=len(state[0])
  display(state)
  while(1):
    print("Column numbers: 0 to 5")
    fromCol=int(input("Enter the column number from which you want to move?"))
    toCol=int(input("Enter the column number to which you want to move?"))
    if fromCol<0 or fromCol>=numberOfCols or toCol<0 or toCol>=numberOfCols:
      print("Enter valid column numbers!!!!")
    else:
      stateColumnWise=convertColumnWise(state)
      if checkIfMoveIsPossible(stateColumnWise,fromCol,toCol)==False:
        print("Invalid move")
      else:
        print("Valid move")
        '''
        Logic for valid move :)    
        '''
        blankSpaces=getNumberOfBlankSpacesInDestCol(stateColumnWise[toCol])
        topEle=getTopElement(stateColumnWise[fromCol])
        numberOfBallsToMove=getNumberOfBallsToMove(stateColumnWise[fromCol],topEle[1],topEle[0])
        if blankSpaces>numberOfBallsToMove:
          # Change - numberOfBallsToMove
          state=makeMove(state,numberOfBallsToMove,topEle[0],fromCol,blankSpaces-1,toCol)
        else:
          # Change - blankSpaces
          state=makeMove(state,blankSpaces,topEle[0],fromCol,blankSpaces-1,toCol)
        moveCount+=1
        display(state)
        if doesPlayerwin(state)==True:
          print("You won the match with {} moves!!!".format(moveCount))
          return moveCount
        if outOfMoves(state)==True:
          print("You Lost the match :)")
          return -1

puz=Puzzle()
number_of_moves=sortPuzzle()
optimal_moves=puz.getOptimalNoOfMoves()
if number_of_moves<=optimal_moves:
  print("You won the match with minimum number of moves")
else:
  print("Optimal Number of moves : ",optimal_moves)
  print("Moves : ")
  puz.printOptimalWays()



- 1 - - - - 
2 3 - 4 - 4 
2 3 - 2 3 4 
1 2 1 1 3 4 
Column numbers: 0 to 5
Enter the column number from which you want to move?1
Enter the column number to which you want to move?2
Valid move
- - - - - - 
2 3 - 4 - 4 
2 3 1 2 3 4 
1 2 1 1 3 4 
Column numbers: 0 to 5
Enter the column number from which you want to move?1
Enter the column number to which you want to move?4
Valid move
- - - - 3 - 
2 - - 4 3 4 
2 - 1 2 3 4 
1 2 1 1 3 4 
Column numbers: 0 to 5
Enter the column number from which you want to move?3
Enter the column number to which you want to move?5
Valid move
- - - - 3 4 
2 - - - 3 4 
2 - 1 2 3 4 
1 2 1 1 3 4 
Column numbers: 0 to 5
Enter the column number from which you want to move?0
Enter the column number to which you want to move?1
Valid move
- - - - 3 4 
- 2 - - 3 4 
- 2 1 2 3 4 
1 2 1 1 3 4 
Column numbers: 0 to 5
Enter the column number from which you want to move?3
Enter the column number to which you want to move?1
Valid move
- 2 - - 3 4 
- 2 - - 3 4 
- 2 1 - 3 4 
1

In [18]:
puz1=Puzzle()
number_of_moves=sortPuzzle()
optimal_moves=puz1.getOptimalNoOfMoves()
if number_of_moves<=optimal_moves:
  print("You won the match with minimum number of moves")
else:
  print("Optimal Number of moves : ",optimal_moves)
  print("Moves : ")
  puz1.printOptimalWays()


- 1 - - - - 
2 3 - 4 - 4 
2 3 - 2 3 4 
1 2 1 1 3 4 
Column numbers: 0 to 5
Enter the column number from which you want to move?1
Enter the column number to which you want to move?2
Valid move
- - - - - - 
2 3 - 4 - 4 
2 3 1 2 3 4 
1 2 1 1 3 4 
Column numbers: 0 to 5
Enter the column number from which you want to move?5
Enter the column number to which you want to move?3
Valid move
- - - 4 - - 
2 3 - 4 - - 
2 3 1 2 3 4 
1 2 1 1 3 4 
Column numbers: 0 to 5
Enter the column number from which you want to move?4
Enter the column number to which you want to move?5
Invalid move
Column numbers: 0 to 5
Enter the column number from which you want to move?3
Enter the column number to which you want to move?5
Valid move
- - - - - 4 
2 3 - - - 4 
2 3 1 2 3 4 
1 2 1 1 3 4 
Column numbers: 0 to 5
Enter the column number from which you want to move?0
Enter the column number to which you want to move?3
Valid move
- - - 2 - 4 
- 3 - 2 - 4 
- 3 1 2 3 4 
1 2 1 1 3 4 
Column numbers: 0 to 5
Enter the colum

#### Sample data



In [ ]:
'''

[['-','1','-','-','-','-'],
         ['2','3','-','4','-','4'],
         ['2','3','-','2','3','4'],
         ['1','2','1','1','3','4']
         ]


['2','3','3','1','-','-'],
          ['2','4','1','4','-','-'],
          ['2','2','1','4','-','-'],
         ['1','3','3','4','-','-']

'''

#### Helpers: 

In [ ]:
# def h(self,start):
#     col=list(zip(*start.data))
#     count=0
#     for i in range(0,self.n):
#       temp_list = [x for x in col[i] if x != '-']
#       temp=set(temp_list)
#       temp=list(temp)
#       count+=len(temp)
#     return count




# class Node:

#   def __init__(self,data,level,fval):
#     self.data=data
#     self.level=level
#     self.fval=fval
#     self.rows=len(data)
#     self.cols=len[]
  
#   def getTopEle(self,column):
#     i=0
#     while i<4 and column[i]=='-':
#       i+=1
#     if i==4:
#       return -1
#     else:
#       return i
  
#   def copy(self,root):
#     temp=[]
#     for i in root:
#       t=[]
#       for j in i:
#         t.append(j)
#       temp.append(t)
#     return temp

#   def shuffle(self,root,noOfEle,srcIndex1,srcIndex2,destIndex1,destIndex2):
#     temp_puz=[]
#     temp_puz=self.copy(root)
#     # print(temp_puz)
#     # print(srcIndex1,srcIndex2)
#     for i in range(0,noOfEle):
#       temp=temp_puz[srcIndex1][srcIndex2]
#       temp_puz[srcIndex1][srcIndex2]=temp_puz[destIndex1][destIndex2]
#       temp_puz[destIndex1][destIndex2]=temp
#       srcIndex1-=1
#       destIndex1+=1

#     return temp_puz

#   def printMatrix(self,data):
#     for i in data:
#       for j in i:
#         print(j,end=" ")
#       print("\n")

#   def gettingCountOfSimiliarEle(self,column,topEle,pos):
#     count=0
#     for i in range(pos,4):
#       if(column[i]==topEle):
#         count+=1
#       else:
#         return count
#     return count

#   def generate_child(self):
#     children=[]
#     col=list(zip(*self.data))
#     for i in range(0,6):
#       topElePos=self.getTopEle(col[i])
#       # print("For {}".format(i))
#       if topElePos!=-1:
#         numberOfEle=self.gettingCountOfSimiliarEle(col[i],col[i][topElePos],topElePos+1)+1
#         # print(numberOfEle)
#         for j in range(0,6):
#           if i!=j:
#             k=0
#             while k<4 and col[j][k]=='-':
#               k=k+1
#             if k!=0 and (k==4 or col[j][k]==col[i][topElePos]):
#               if k>=numberOfEle:
#                 child=self.shuffle(self.data,numberOfEle,k-1,j,topElePos,i)
#               else:
#                 child=self.shuffle(self.data,k,k-1,j,topElePos,i)
#               # self.printMatrix(self.data)
#               # print("\nPrinting child:\n")
#               # self.printMatrix(child)
#               child_node=Node(child,self.level-1,0)
#               children.append(child_node)
#     return children
              
        

In [ ]:
# class Puzzle:
#   def __init__(self):
#     self.n=0
#     self.open=[]
#     self.closed=[]

#   def h(self,start):
#     col=list(zip(*start.data))
#     count=0
#     for i in range(0,self.n):
#       temp_list = [x for x in col[i] if x != '-']
#       temp=set(temp_list)
#       temp=list(temp)
#       count+=len(temp)
#     return count

#   def f(self,start):
#     return start.level+self.h(start)
  
#   def returnBest(self,lists):
#     countList=[]
#     for i in lists:
#       count=0
#       cols=list(zip(*i.data))
#       for j in range(0,self.n):
#         temp_list = [x for x in cols[j] if x != '-']
#         temp=set(temp_list)
#         temp=list(temp)
#         count+=(len(temp_list)+1-len(temp))
#       countList.append(count)
#     max_index=countList.index(max(countList))
#     return max_index
#     # print(max_index)
#     # lists[max_index].printMatrix(lists[max_index].data)
#     # print(countList)

#   def accept(self):
#     puz=[['3','2','1','2','-','-'],
#           ['2','3','1','1','-','-'],
#           ['1','2','3','3','-','-']
#          ]
#     self.size=len(puz[0])
#     # for i in range(0,self.n):
#     #   temp = input().split(" ")
#     #   puz.append(temp)
#     return puz

#   def process(self):
#     start=self.accept()
#     start=Node(start,0,0)
#     start.fval=self.h(start)
#     print(start.fval)
#     self.open.append(start)
#     moves=-1
#     while True:
#         moves+=1
#         cur = self.open[0]
#         cur.printMatrix(cur.data)
#         # print("\t|\n\t\/")
#         if(self.h(cur) == 4):
#           print("No of moves:",moves)
#           break
#         d=cur.generate_child()
#         for i in d:
#           i.fval = self.f(i)
#           self.open.append(i)
#         self.closed.append(cur)
#         del self.open[0]
#         self.open.sort(key = lambda x:x.fval,reverse=False)
#         first_fval=self.open[0].fval
#         lists=[i for i in self.open if i.fval==first_fval]
#         max_index=self.returnBest(lists)
#         temp=self.open[0]
#         self.open[0]=self.open[max_index]
#         self.open[max_index]=temp
#         print("Next child")
#         self.open[0].printMatrix(self.open[0].data)
#         print(self.open[0].fval)
#         print("---------------------------------------------------------------")
